# Cisco - Ariel University API Security Detection Challenge 2023
## Baseline code


### Imports and global settings

In [1]:
# Imports, settings and first dataset view
import pandas as pd
import seaborn as sns
import numpy as np
import json
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler 
import re


LOG_4_ATTACK = "LOGJATTACKHERE"

# Set pandas to show all columns when you print a dataframe
pd.set_option('display.max_columns', None)

# Global setting here you choose the dataset number and classification type for the model
dataset_number = 2 # Options are [1, 2, 3, 4]
test_type = 'label' # Options are ['label', 'attack_type']


# Read the json and read it to a pandas dataframe object, you can change these settings
with open(f'./dataset_{str(dataset_number)}_train.json') as file:
    raw_ds = json.load(file)
    file.close()
df = pd.json_normalize(raw_ds, max_level=2)


# Shoe the first five lines of the dataframe to see if everything was read accordingly 
df.head()


KeyboardInterrupt



### Basic dataset label arrangements

In [ ]:
# Fill the black attack tag lines with "Benign" string
df['request.Attack_Tag'] = df['request.Attack_Tag'].fillna('Benign')
df['attack_type'] = df['request.Attack_Tag']

# This function will be used in the lambda below to iterate over the label columns 
# You can use this snippet to run your own lambda on any data with the apply() method
def categorize(row):  
    if row['request.Attack_Tag'] == 'Benign':
        return 'Benign'
    return 'Malware'

df['label'] = df.apply(lambda row: categorize(row), axis=1)

# df[df['attack_type'] != 'SQL Injection'][['request.url']] = df[df['attack_type'] != 'SQL Injection'][['request.url']].apply(lambda x: x.replace('http://127.0.0.1:5000/', ''))

# After finishing the arrangements we delete the irrelevant column
df.drop('request.Attack_Tag', axis=1, inplace=True)

df.head()

In [ ]:
def fill_na_df(df_na):
    # Remove all NAN columns or replace with desired string
    # This loop iterates over all of the column names which are all NaN
    for column in df_na.columns[df_na.isna().any()].tolist():
        # df.drop(column, axis=1, inplace=True)
        df_na[column] = df_na[column].fillna('None')

    # If you want to detect columns that may have only some NaN values use this:
    #df.loc[:, df.isna().any()]

fill_na_df(df)
df
df_checker = pd.DataFrame(df)

In [ ]:
keep_list = ['Benign', 'SQL Injection', 'RCE']
f = df[df['attack_type'].isin(keep_list)]
f = df[df['attack_type'] == 'LOG4J']
#print(*f['request.url'].values.tolist(), sep='\n')
set_cols = set()
counter = 0
column_names = f.columns.values.tolist()
for index, row in f.iterrows():
    for col in column_names:
        data = str(row[col])
        if re.search(r'\$\{[a-z]+:.*\}', data):
            print(f'index:\t{index}')
            print(f'col name:\t{col}')
            print(f'data:{data}\n')
            counter += 1
            set_cols.add(col)
print()
print(set_cols)
print('counter:', counter)
f.shape

In [ ]:
#try erea
special_rep = {'{}': 'EMPTYDIR'}
special_str = ['[', ']', '--', ';', "'", '{', '}', '=', ',']
patterns_rep = {r'\%\d\d': r' \g<0> '}

def replace_str_for_types(s):
    for k in special_rep:
        s = s.replace(k, special_rep[k])
    for i in special_str:
        s = s.replace(i, f' {i} ')
    for patt in patterns_rep:
        s = re.sub(patt, patterns_rep[patt], s)
    return s

def space_words_data(s):
    return re.sub(r'(\%\d\d)|([^A-Za-z0-9-_])', r' \g<0> ', s)

def is_sql_type(s: str):
    patterns = [r"';SELECT%20\*%20FROM%20.*(--)?", # select * from...
                r"'.*--",
                r"'%20or%20'(.*)'.*=.*'\1.*"]      # is match to the format of 'or 1=1'
    for patt in patterns:
        if re.search(patt, s):
            return True
    return False

r_s = "orders/get/country?country=';SELECT%20*%20FROM%20order_details%20--"
print(space_words_data(r_s))

# keep_list = ['Benign', 'Cookie Injection', 'Directory Traversal', 'Log Forging']
# keep_list = ['Benign', 'LOG4J']#, 'SQL Injection']
# df = df[df['attack_type'].isin(keep_list)]
#print(*f[['request.url']].values.tolist(), sep='\n')
info= {"Num":[12,14,13,12,14,13,15], "NAME":['John','Camili','Rheana','Joseph','Amanti','Alexa','Siri']}
 
data = pd.DataFrame(info)
print(data)
data.iloc[0]='new value'



In [ ]:
# f = df[(df['attack_type'] == "SQL Injection")]
# l = []
# column_names = f.columns.to_list()
#     for index, row in df.iterrows():
#         if row['request.url']

In [ ]:
# Setting features for further feature extraction by choosing columns
# Some will be "simply" encoded via label encoding and others with HashingVectorizer

LOG4_COL_NAME = 'isLog4J'
# SQLI_COL_NAME = 'isSQLI'


# On these headers we will run a "simple" BOW
SIMPLE_HEADERS = ['request.headers.Accept-Encoding',
                'request.headers.Connection',
                'request.headers.Host',
                'request.headers.Accept',
                'request.method',
                'request.headers.Accept-Language',
                'request.headers.Sec-Fetch-Site',
                'request.headers.Sec-Fetch-Mode',
                'request.headers.Sec-Fetch-Dest',
                'request.headers.Sec-Fetch-User',
                'response.status',
                    'response.headers.Content-Length',
                  LOG4_COL_NAME
                ]

# On these headers we will run HashingVectorizer
COMPLEX_HEADERS = ['request.headers.User-Agent',
                   'request.headers.Set-Cookie',
                   'request.headers.Date',
                   'request.url',
                   'response.headers.Content-Type',
                   'response.body',
                   'response.headers.Location',
                   'request.headers.Content-Length',
                   'request.headers.Cookie',
                   'response.headers.Set-Cookie'
                   
                    ]

COLUMNS_TO_REMOVE = ['request.body',
                    'request.headers.Date']

SQL_LABEL = 'ItIsSQLI'
# This is our main preprocessing function that will iterate over all of the chosen 
# columns and run some feature extraction models
def vectorize_df(df):
    #remove the url
#     df['request.url'] = df['request.url'].apply(lambda x: x.replace('http://127.0.0.1:5000/', ''))
    df['request.url'] = df['request.url'].apply(lambda x: SQL_LABEL if is_sql_type(x) else x)
    
    column_names = df.columns.to_list()
    df[LOG4_COL_NAME] = 'reg'
    for index, row in df.iterrows():
        for col in column_names:
            data = df.loc[index, col]
            if type(data) is not str:
                continue
#             if col == 'request.url' and is_sql_type(data):
#                 df.loc[index] = '0'
#                 df.loc[index, 'attack_type'] = 'SQL Injection'
#                 df.loc[index, 'label'] = 'Malware'
#                 break
            if re.search(r'\$\{[a-z]+:.*\}$', data):
                df.loc[index, LOG4_COL_NAME] = LOG_4_ATTACK # isLog4J
                break
    
#     for column in column_names:
#         df[column] = df[column].apply(lambda x: replace_str_for_types(str(x)))
# #         print(df[column])
        
    le = LabelEncoder()
    h_vec = HashingVectorizer(n_features=4, ngram_range=(1, 4), alternate_sign=False, token_pattern=r'[^\s/]+') #, ngram_range=(1, 4)
#     h_vec = HashingVectorizer(n_features=20, ngram_range=(1, 10), alternate_sign=False, token_pattern=r'[^\s/]+') #, ngram_range=(1, 4)
    # Run LabelEncoder on the chosen features
    for column in SIMPLE_HEADERS: 
        df[column] = le.fit_transform(df[column])
    
    # Run HashingVectorizer on the chosen features
    for column in COMPLEX_HEADERS: 
#         df[column].apply(space_words_data)
        newHVec = h_vec.fit_transform(df[column])
        df[column] = newHVec.todense()

    # Remove some columns that may be needed.. (Or not, you decide)
    for column in COLUMNS_TO_REMOVE: 
        df.drop(column, axis=1, inplace=True)
    return df

df = vectorize_df(df)
df

In [ ]:
# Memory check (For large datasets sometimes the dataframe will exceed the computers resources)
df.info(memory_usage="deep")

In [ ]:
# Choose the right features
# In our example code we choose all the columns as our feature this can be the right or wrong way to approach the model, you choose.

features_list = df.columns.to_list()
features_list.remove('label')
features_list.remove('attack_type')

# the first (include a lot)
# black_list = ['request.headers.Connection',
#                 'request.headers.Host',
#                 'request.headers.Accept',
#                 'request.headers.Sec-Fetch-User'] # good for cookiem directory, log-forg

# for feat in black_list:
#     features_list.remove(feat)
    
# feature_list = ['response.headers.Content-Type',
#                 'response.body',
#                 'response.status_code',
#                 'response.status',
#                 'response.headers.Location']

# features_list = ['request.headers.Sec-Fetch-Dest',
#               'request.headers.Accept-Encoding',
#               'request.headers.Sec-Fetch-Site',
#               'request.headers.Set-Cookie']

# features_list = ['request.url',
#                 'response.body',
#                 'response.status_code',
#                 'response.status', 'request.headers.Set-Cookie', LOG4_COL_NAME]
print(features_list)

# Recheck all datatype before training to see we don't have any objects in our features
# In this example our model must get features containing only numbers so we recheck to see if we missed anything during preprocessing
df.dtypes

In [ ]:
# Increase the size of the heatmap.
plt.figure(figsize=(16, 6))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12});

## Train test split

In [ ]:
# Data train and test split preparations. Here we will insert our feature list and label list.
# Afterwards the data will be trained and fitted on the amazing XGBoost model
# X_Train and y_Train will be used for training
# X_test and y_test.T will be used for over fitting checking and overall score testing

# We convert the feature list to a numpy array, this is required for the model fitting
X = df[features_list].to_numpy()

# This column is the desired prediction we will train our model on
y = np.stack(df[test_type])

# We split the dataset to train and test according to the required ration
# Do not change the test_size -> you can change anything else
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1765, random_state=42, stratify=y)

# We print the resulted datasets and count the difference 
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
counter = Counter(y)
counter


## Model choosing and fitting

In [ ]:
sub_features_list = ['request.url'
                'response.body']

sub_label_list = ['Benign', 'RCE']

def get_sub_train(X, y, clf, feat_og, feat_sub, label_sub, isNeg=False):
    if label_sub is None:
        label = []
    bool_feats = np.isin(feat_og, feat_sub)
    bool_labels = np.isin(y, label_sub)
    if isNeg:
        bool_labels = ~bool_labels
    X_new = X[bool_labels][:, bool_feats]
    y_new = y[bool_labels]
    
    return (X_new , y_new, clf.fit(X_new , y_new))

In [ ]:
######################## tryyyyy ######################

from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
# from sklearn.preprocessing import StandardScaler

# We choose our model of choice and set it's hyper parameters you can change anything
clf = DecisionTreeClassifier()
# Train Model
# clf2.fit(X_train_1, y_train_1)
clf.fit(X_train, y_train)

X_train_1, y_train_1, clf1 = get_sub_train(X_train, y_train, 
                                           RandomForestClassifier(n_estimators=50),
                                           features_list, features_list, sub_label_list)

clf_feat_label_list = [(clf1, features_list, sub_label_list)]


# clf2.fit(X_train_1, y_train_1)

# Check data balance and variety
print("train:")
print(*sorted(Counter(y_train).items()), sep='\n')

print()

print("test labels:")
print(*sorted(Counter(y_test).items()), sep='\n')

######################## tryyyyy ######################

## Result printout

In [ ]:
def predict(data, clf, clf_feat_label_list):
    predictions = clf.predict(data)
    
    for clf2, feats, labels in clf_feat_label_list:
        bool_feats = np.isin(features_list, feats)
        data_new = data[:, bool_feats]
        for i in range(predictions.shape[0]):
            if predictions[i] in labels:
                predictions[i] = clf2.predict([data_new[i]])[0]
    
    return predictions


In [ ]:
# We print our results
predictions = predict(X_test, clf, clf_feat_label_list) # clf.predict(X_test) #

def value_accu(predictions, y_test):
    sns.set(rc={'figure.figsize':(15,8)})

    true_labels = y_test

    cf_matrix = confusion_matrix(predictions, true_labels)
    clf_report = classification_report(true_labels, predictions, digits=5)
    heatmap = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g', 
                          xticklabels=np.unique(true_labels), 
                          yticklabels=np.unique(true_labels)) 

    # The heatmap is cool but this is the most important result
    print(clf_report)
    
value_accu(predictions, y_test)

In [ ]:
# h = zip(predictions, true_labels)
# counter = y_train.shape[0] + 1
# # print(*h, sep='\n')
# dict_data = {}
# for p,t in h:
#     s = f'{p}->{t}'
#     if s not in dict_data:
#         dict_data[s] = []
#     dict_data[s] += [counter]
#     counter += 1
# print(*dict_data, sep='\n')
# print('\n')

# # print('Benign->SQL Injection:', dict_data['Benign->SQL Injection'], sep='\n')

# # print('SQL Injection->Benign', dict_data['SQL Injection->Benign'], sep='\n')
# # print()
# # print('fake SQLI:')
# ind_list = dict_data['Benign->Malware']

# df_checker.iloc[ind_list]

# Test

In [ ]:
# Now it's your turn, use the model you have just created :)
raw_ds = {}

# Read the valuation json, preprocess it and run your model 
with open(f'./dataset_{str(dataset_number)}_val.json') as file:
    raw_ds = json.load(file)

test_df = pd.json_normalize(raw_ds, max_level=2)

fill_na_df(test_df)
    
test_df

# Preprocess the validation dataset, remember that here you don't have the labels
test_df = vectorize_df(test_df)

# Predict with your model
X = test_df[features_list].to_numpy()
predictions = predict(X_test, clf, clf_feat_label_list) # clf.predict(X)

predictions

# Save your predictions

In [ ]:
# Save your preditions
enc = LabelEncoder()

np.savetxt(f'./dataset_{str(dataset_number)}_{test_type}_result.txt', enc.fit_transform(predictions), fmt='%2d')

In [ ]:
# print(*df_checker[df_checker['attack_type'] == "SQL Injection"]['request.url'].values.tolist(), sep='\n')

In [ ]:
X_train

In [3]:
l = []
with open("dataset_3_attack_type_result.txt", 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line != " 0\n":
            l += [" 1\n"]
        else:
            l += [" 0\n"]
print(lines)

with open("dataset_3_label_result.txt", 'w') as f:
    f.writelines(l)

[' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 1\n', ' 0\n', ' 0\n', ' 0\n', ' 1\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 1\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 4\n', ' 0\n', ' 0\n', ' 1\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 3\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 2\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 4\n', ' 2\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 0\n', ' 3\n', ' 3\n',